In [8]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


Affinity reported from Win32 API: 0, 1, 2, 3, 4, 5, 6, 7;
Affinity reported from Win32 API: 0, 1, 2, 3, 4, 5, 6, 7;
Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Databases loaded: 
Capacity: 0
Count: 0



Error: System.ApplicationException: Already called.
   at BoSSS.Application.BoSSSpad.BoSSSshell.InitTraceFile() in C:\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 217
   at BoSSS.Application.BoSSSpad.BoSSSshell.Init() in C:\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 104
   at Submission#9.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

init workflow


In [9]:
BoSSSshell.WorkflowMgm.Init("XESF_BowShock_OptProbStudy");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


Project name is set to 'XESF_BowShock_OProbStudy2'.
Default Execution queue is chosen for the database.
Opening existing database '\\dc3\userspace\sebastian\cluster\XESF_BowShock_OProbStudy2'.


{ Session Count = 6; Grid Count = 6; Path = \\dc3\userspace\sebastian\cluster\XESF_BowShock_OProbStudy2 }

In [10]:
var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var database = OpenOrCreateDatabase(dbPath);
var dt = database.Sessions.Last().CreationTime;
dt


2024-03-12 14:03:46Z

In [11]:
var sessions = database.Sessions;
sessions

#0: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iProb1-iFlux0-FphiType0-aRI_True	3/12/2024 2:04:06 PM	931567aa...
#1: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iProb2-iFlux1-FphiType0-aRI_True	3/12/2024 2:05:08 PM	67852b92...
#2: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iProb2-iFlux0-FphiType0-aRI_True	3/12/2024 2:04:47 PM	ea0cf466...
#3: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-aRI_True	3/12/2024 2:03:23 PM	42574efb...
#4: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iProb1-iFlux1-FphiType0-aRI_True	3/12/2024 2:04:25 PM	c1b92564...
#5: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iProb0-iFlux1-FphiType0-aRI_True	3/12/2024 2:03:46 PM	2f3f589b...


In [12]:
MatrixAssembler aaa = new MatrixAssembler(3);
var si = database.Sessions.Pick(0);
var ti = si.Timesteps;
si.ID

931567aa-f307-492e-95c5-d1cb419031cb

## Enthalpy error
enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(c_P -1)(\rho E - k),~k=c_P \text{Ma}^2\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$


For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

In the case of the BowShock presented, the v-velocity is zero and we have
$$ h_{in} = \frac{\gamma}{\gamma-1} + \frac{1}{2}u_{in}^2 $$
We prescribe the inflow conditions in terms of pressure $p_{in}=1$, density $\rho_{in}=1$ and the Mach number $M_{in}=4$. From those the velocity is computed as 
$$ u_{in} = M_{in} \sqrt{\gamma \frac{p_{in}}{\rho_{in}}}= 4\sqrt{\gamma}$$

In [13]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = (XDGField) si.Timesteps.Last().GetField("h");
var enthalpy_exact = (XDGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 4;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

14.7

In [14]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.GetSpeciesShadowField("L").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.GetSpeciesShadowField("R").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2NormAllSpecies();

## Choose which runs to plot

In [20]:
var ses=sessions.Where(s => (s.SuccessfulTermination )); // the one plotted in the publication
ses

#0: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iProb1-iFlux0-FphiType0-aRI_True	3/12/2024 2:04:06 PM	931567aa...
#1: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iProb2-iFlux1-FphiType0-aRI_True	3/12/2024 2:05:08 PM	67852b92...
#2: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iProb2-iFlux0-FphiType0-aRI_True	3/12/2024 2:04:47 PM	ea0cf466...
#3: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-aRI_True	3/12/2024 2:03:23 PM	42574efb...
#4: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iProb1-iFlux1-FphiType0-aRI_True	3/12/2024 2:04:25 PM	c1b92564...
#5: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iProb0-iFlux1-FphiType0-aRI_True	3/12/2024 2:03:46 PM	2f3f589b...


# A Helper Function to obtain different plot formats

In [21]:
using BoSSS.Solution.Tecplot;
using System.IO;
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

## Actual Etnhalpy Error Plot


In [22]:
int count=0;
foreach(var si in ses){
    var enthalpyerrors = new List<double>();
    foreach(var timestep in si.Timesteps){
        var error = ((XDGField) timestep.GetField("h_err")).L2NormAllSpecies()/EEN;
        enthalpyerrors.Add(error);
    }
    allEE.Add(enthalpyerrors);
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Pick(enthalpyerrors.Count -1)).GetInternal();
    plot.AddDataGroup(si.Name,lts.TimeStepNumbers,enthalpyerrors, GetFormat(count));
    count++;
    enthalpyerrors.SaveToTextFile($"EnthalpyErrors_{si.Name}.txt");
}
plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:600)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 1 
 
 
 
 
 10 2 
 
 
 
 
 10 3 
 
 
 
 
 10 4 
 
 
 
 
 0 
 
 
 
 
 50 
 
 
 
 
 100 
 
 
 
 
 150 
 
 
 
 
 200 
 
 
 
 
 250 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 XDGBS-p3-10x32-agg0.4-iProb1-iFlux0-FphiType0-aRITrue 
 
 
 XDGBS-p3-10x32-agg0.4-iProb1-iFlux0-FphiType0-aRI T rue 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M500.1,57.1 L553.5,57.1 M45.6,366.2 L50.0,359.6 L54.4,361.3 L58.9,361.6 L63.3,361.7 L67.7,361.7
 L72.1,361.7 L76.5,361.7 L80.9,361.7 L85.4,361.7 L89.8,361.7 L94.2,361.7 L98.6,361.7 L103.0,363.0
 L107.5,363.6 L111.9,363.8 L116.3,364.5 L120.7,365.6 L125.1,365.7 L129.5,365.6 L134.0,365.8 L138.4,365.8
 L142.8,366.1 L147.2,366.4 L151.6,366.3 L156.0,366.5 L160.5,366.5 L164.9,366.4 L169.3,366.3 L173.7,366.3
 L178.1,366.3 L182.6,366.3 L187.0,366.3 L191.4,366.3 L195.8,366.4 L200.2,366.4 L204.6,366.4 L209.1,366.4
 L213.5,366.4 L217.9,366.4 L222.3,374.4 L226.7,377.0 M235.6,384.9 L240.0,392.5 L244.4,415.0 L248.8,421.9
 L253.2,434.9 L257.7,425.2 L262.1,431.6 L266.5,440.3 L270.9,440.5 L275.3,440.3 L279.8,440.5 L284.2,440.4
 L288.6,440.7 L293.0,440.3 L297.4,440.7 L301.8,440.3 L306.3,440.7 L310.7,440.5 L315.1,440.7 L319.5,440.3
 L323.9,440.7 L328.4,440.3 L332.8,440.7 L337.2,440.5 L341.6,440.6 L346.0,440.6 L350.4,440.5 L354.9,440.6
 L359.3,440.5 L363.7,440.5 L368.1,440.6 L372.5,440.5 L376.9,440.5 L381.4,440.3 L385.8,440.5 L390.2,440.1
 L394.6,440.5 L399.0,440.5 L403.5,440.5 L407.9,477.1 L412.3,478.1 L416.7,480.3 L421.1,480.4 L425.5,479.8
 L430.0,479.1 L434.4,479.0 L438.8,479.3 L443.2,480.1 L447.6,480.3 L452.1,480.3 L456.5,479.9 L460.9,479.9
 L465.3,480.1 L469.7,480.2 L474.1,480.2 L478.6,480.3 L483.0,480.3 L487.4,480.4 L491.8,480.4 L496.2,480.4
 L500.7,480.4 L505.1,480.4 L509.5,480.4 L513.9,480.4 L518.3,480.4 L522.7,480.4 L527.2,480.4 L531.6,480.4
 L536.0,480.4 L540.4,480.4 L544.8,480.4 L549.3,480.4 L553.7,480.4 L558.1,480.4 L562.5,480.4 L566.9,480.4
 L571.3,480.4 L575.8,480.4 L580.2,480.4 L584.6,480.4 L589.0,480.4 L593.4,480.4 L597.8,480.4 L602.3,480.4
 L606.7,480.4 L611.1,480.4 L615.5,480.4 L619.9,480.4 L624.4,480.4 L628.8,480.4 L633.2,480.4 L637.6,480.0
 L642.0,480.4 L646.4,480.4 L650.9,480.4 L655.3,480.4 L659.7,480.4 L664.1,480.4 L668.5,480.4 L673.0,480.4
 L677.4,480.4 L681.8,510.5 L686.2,512.0 L690.6,512.0 L695.0,512.1 L699.5,512.1 L703.9,512.1 L708.3,512.0
 L712.7,512.0 L717.1,512.0 L721.6,512.0 L726.0,512.0 L730.4,512.0 L734.8,512.0 L739.2,512.1 L743.6,510.5
 L748.1,512.1 L752.5,512.1 L756.9,512.0 L761.3,512.1 L765.7,512.1 L770.2,512.1 L774.6,512.1 L779.0,512.1
 L783.4,512.1 L787.8,512.1 L792.2,512.1 L796.7,512.1 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGBS-p3-10x32-agg0.4-iProb2-iFlux1-FphiType0-aRITrue 
 
 
 XDGBS-p3-10x32-agg0.4-iProb2-iFlux1-FphiType0-aRI T rue 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M500.1,81.1 L553.5,81.1 M45.6,366.2 L50.0,365.5 L54.4,365.3 L58.9,365.2 L63.3,364.7 L67.7,364.7
 L72.1,364.8 L76.5,364.8 L80.9,364.8 L85.4,364.8 L89.8,364.5 L94.2,363.8 L98.6,363.6 L103.0,363.2
 L107.5,363.5 L111.9,363.8 L116.3,363.2 L120.7,363.5 L125.1,363.6 L129.5,363.7 L134.0,363.9 L138.4,364.6
 L142.8,365.4 L147.2,365.9 L151.6,366.0 L156.0,366.1 L160.5,366.1 L164.9,366.1 L169.3,366.1 L173

## Residual History

In [23]:

using BoSSS.Solution.Tecplot;
using System.IO;
//Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
int count=0;
foreach(var si in ses){
    var enthalpyerrors = new List<double>();
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    plot.AddDataGroup("||r|| - "+ si.Name,lts.TimeStepNumbers,lts.ResHistory, GetFormat(count));
    lts.ResHistory.SaveToTextFile($"ResHistory_{si.Name}.txt");
    count++;
    plot.AddDataGroup("EnRes_"+si.Name,lts.TimeStepNumbers,lts.EnResHistory, GetFormat(count));
    lts.EnResHistory.SaveToTextFile($"EnResHistory_{si.Name}.txt");
    count++;
}

plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:800,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -6 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 1 
 
 
 
 
 10 2 
 
 
 
 
 10 3 
 
 
 
 
 10 4 
 
 
 
 
 0 
 
 
 
 
 50 
 
 
 
 
 100 
 
 
 
 
 150 
 
 
 
 
 200 
 
 
 
 
 250 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||r|| - XDGBS-p3-10x32-agg0.4-iProb1-iFlux0-FphiType0-aRITrue 
 
 
 ||r|| - XDGBS-p3-10x32-agg0.4-iProb1-iFlux0-FphiType0-aRI T rue 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M11.3,57.1 L64.7,57.1 M45.6,105.7 L48.4,111.0 L51.2,113.4 L54.1,114.0 L56.9,114.0 L59.7,114.0
 L62.5,114.0 L65.3,114.0 L68.1,114.0 L71.0,114.0 L73.8,114.0 L76.6,114.0 L79.4,114.0 L82.2,114.8
 L85.1,116.5 L87.9,117.0 L90.7,117.8 L93.5,117.8 L96.3,120.0 L99.1,127.1 L102.0,132.9 L104.8,134.9
 L107.6,147.6 L110.4,154.3 L113.2,154.3 L116.0,167.9 L118.9,169.3 L121.7,164.9 L124.5,167.9 L127.3,161.1
 L130.1,168.6 L133.0,160.2 L135.8,175.3 L138.6,161.7 L141.4,174.3 L144.2,162.1 L147.0,172.7 L149.9,162.0
 L152.7,165.9 L155.5,172.9 L158.3,108.2 L161.1,117.5 L164.0,119.3 L166.8,123.8 L169.6,118.3 L172.4,125.2
 L175.2,149.0 L178.0,143.6 L180.9,176.6 L183.7,180.9 L186.5,186.1 L189.3,195.8 L192.1,186.3 L195.0,192.2
 L197.8,185.6 L200.6,193.0 L203.4,188.8 L206.2,193.9 L209.0,190.2 L211.9,195.2 L214.7,194.1 L217.5,196.2
 L220.3,191.5 L223.1,194.9 L226.0,197.0 L228.8,198.3 L231.6,195.8 L234.4,195.7 L237.2,193.4 L240.0,198.2
 L242.9,199.8 L245.7,199.6 L248.5,200.1 L251.3,200.7 L254.1,206.5 L256.9,202.6 L259.8,205.6 L262.6,188.0
 L265.4,191.7 L268.2,201.0 L271.0,192.6 L273.9,193.5 L276.7,162.5 L279.5,159.7 L282.3,198.2 L285.1,198.1
 L287.9,205.1 L290.8,202.6 L293.6,206.6 L296.4,210.1 L299.2,210.4 L302.0,212.0 L304.9,215.1 L307.7,217.9
 L310.5,219.9 L313.3,221.4 L316.1,224.7 L318.9,225.7 L321.8,227.6 L324.6,229.0 L327.4,228.9 L330.2,229.2
 L333.0,229.4 L335.9,230.2 L338.7,231.0 L341.5,231.8 L344.3,230.3 L347.1,231.2 L349.9,231.4 L352.8,231.9
 L355.6,231.4 L358.4,234.3 L361.2,234.7 L364.0,234.1 L366.9,234.3 L369.7,235.0 L372.5,234.2 L375.3,234.9
 L378.1,235.7 L380.9,235.6 L383.8,237.3 L386.6,237.7 L389.4,237.5 L392.2,237.1 L395.0,237.3 L397.8,238.2
 L400.7,238.4 L403.5,238.7 L406.3,238.7 L409.1,238.5 L411.9,238.6 L414.8,238.9 L417.6,238.9 L420.4,239.0
 L423.2,239.0 L426.0,239.0 L428.8,239.2 L431.7,239.2 L434.5,239.2 L437.3,239.0 L440.1,239.1 L442.9,239.1
 L445.8,239.3 L448.6,239.3 L451.4,201.7 L454.2,217.5 L457.0,218.1 L459.8,232.1 L462.7,232.7 L465.5,239.7
 L468.3,242.1 L471.1,239.8 L473.9,241.0 L476.8,249.9 L479.6,254.8 L482.4,255.0 L485.2,258.8 L488.0,264.4
 L490.8,266.1 L493.7,267.1 L496.5,265.6 L499.3,266.3 L502.1,267.8 L504.9,268.0 L507.8,266.9 L510.6,266.8
 L513.4,267.0 L516.2,265.4 L519.0,267.6 L521.8,267.6 L524.7,267.6 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGBS-p3-10x32-agg0.4-iProb1-iFlux0-FphiType0-aRITrue 
 
 
 EnRes X DGBS-p3-10x32-agg0.4-iProb1-iFlux0-FphiType0-aRI T rue 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M11.3,81.1 L64.7,81.1 M45.6,89.2 L48.4,92.0 L51.2,93.2 L54.1,93.7 L56.9,93.7 L59.7,93.7
 L62.5,93.7 L65.3,93.7 L68.1,93.7 L71.0,93.7 L73.8,93.7 L76.6,93.7 L79.4,93.7 L82.2,95.0
 L85.1,96.0 L87.9,96.3 L90.7,97.2 L93.5,98.2 L96.3,99.9 L99.1,103.0 L102.0,107.0 L104.8,109.5
 L107.6,112.1 L110.4,112.7 L113.2,113.0 L116.0,113.1 L118.9,113.

### Plot ShadowFields

In [ ]:
BoSSS.Solution.Application.DeleteOldPlotFiles();
foreach(var si in ses){
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    var flds=lts.GetShadowFields();
    var tp= new Tecplot(flds.Last().GridDat,2);
    tp.PlotFields(si.Name,0,flds);
}

rm XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0.plt XDGBS-p3-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0.plt XDGBS-p3-10x32-agg0.4-iPrb0-iFlx2-wFLx0-bFlx0-Fphi0.plt XDGBS-p3-10x32-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0.plt;
